# Install Transformers Library

In [2]:
!nvidia-smi

Wed Jul 21 02:33:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install transformers

     |████████████████████████████████| 2.5 MB 33.6 MB/s 
     |████████████████████████████████| 895 kB 51.0 MB/s 
     |████████████████████████████████| 3.3 MB 53.2 MB/s 


In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
# from transformers import AutoModel, BertTokenizerFast
from transformers import RobertaTokenizer, RobertaModel
# specify GPU
device = torch.device("cuda")

In [6]:
df = pd.read_json('/content/drive/MyDrive/sercasm/twitter/sarcasm_detection_shared_task_twitter_training.jsonl',lines=True)
df1 = pd.read_json('/content/drive/MyDrive/sercasm/twitter/sarcasm_detection_shared_task_twitter_testing.jsonl',lines=True)
dfArg_c=pd.read_csv('/content/drive/MyDrive/sarcasm-detection/augumented_data/context1.csv')
dfArg_r=pd.read_csv('/content/drive/MyDrive/sarcasm-detection/augumented_data/response3.csv')
df['labels'] = df['label'].apply(lambda x: ['SARCASM', 'NOT_SARCASM'].index(x))
df1['labels'] = df1['label'].apply(lambda x: ['SARCASM', 'NOT_SARCASM'].index(x))
df1.tail()

,label,context,response,id,labels
1795,NOT_SARCASM,[I have been a business customer of MWeb @USER...,@USER @USER @USER is definitely the best out t...,twitter_1796,1
1796,SARCASM,[A woman refuses to have her temperature taken...,@USER @USER Ye let her out run wild and infect...,twitter_1797,0
1797,SARCASM,[The reason big government wants @USER out is ...,"@USER @USER @USER Thanks for that , I would ha...",twitter_1798,0
1798,NOT_SARCASM,[Happy #musicmonday and #thanks for #all your ...,@USER @USER @USER Yes also #found this on #new...,twitter_1799,1
1799,NOT_SARCASM,[Not long wrapped on the amazing #January22nd ...,@USER @USER @USER you still need to send the l...,twitter_1800,1


In [7]:
from sklearn.utils import shuffle

In [8]:
# df=shuffle(df,random_state=42)
# dfArg_r=shuffle(dfArg_r,random_state=42)
# df=shuffle(df,random_state=422)
# dfArg_r=shuffle(dfArg_r,random_state=422)
# dfArg_r

In [9]:
print(df['response'])

0       @USER @USER @USER I don't get this .. obviousl...
1       @USER @USER trying to protest about . Talking ...
2       @USER @USER @USER He makes an insane about of ...
3       @USER @USER Meanwhile Trump won't even release...
4       @USER @USER Pretty Sure the Anti-Lincoln Crowd...
                              ...                        
4995    @USER You don't . I have purchased a lot on Am...
4996    @USER #Emotions you say 🤔 never knew that I th...
4997    @USER @USER @USER You are so right ... " Yes !...
4998    @USER @USER @USER Another lazy delusional vote...
4999    @USER @USER I hope you know no news outlet fro...
Name: response, Length: 5000, dtype: object


In [10]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm
!pip install demoji
!pip install contractions
import demoji
import contractions

demoji.download_codes()
nltk.download('stopwords') 
# nltk.download('wordnet')

     |████████████████████████████████| 321 kB 34.8 MB/s 
     |████████████████████████████████| 283 kB 56.0 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85448 sha256=9d997c3ba36b8997e37d288971ed56ab3e0ae269f0d66aaa8674d45685384904
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
... OK (Got response in 0.13 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation(https://www.nltk.org/_modules/nltk/tokenize/regexp.html)
    text = re.sub(r"@",'', text)
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)
    text = re.sub(r"USER",'@USER', text)
    # text = re.sub(r"USER",'', text)
    text = re.sub(r'<URL>','',text)

    text = demoji.replace_with_desc(text)
    text = re.sub(r':','',text)
    text = contractions.fix(text)
    # text = re.sub(r'\.+','',text)
    # tokeniser = RegexpTokenizer(r'\s+', gaps=True)
    # tokeniser = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
    # tokeniser = RegexpTokenizer(r'\w+')
    # tokens = tokeniser.tokenize(text)

    # Lowercase and lemmatise 
    # lemmatiser = WordNetLemmatizer()
    # lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    # keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    
    # stemming
    # ps=PorterStemmer()
    # keywords=[ps.stem(x) for x in keywords]
    return text

def preprocess_text2(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation (optional)
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [12]:
def fun(A,l):
  if len(A)<=l:
    return A
  else:
    # return []#A[-l::1]
    return A[-1:-l-1:-1]
def fun1(A):
  return ''.join(A)

In [13]:
l=400



dfArg_r['tweets']=  dfArg_r['response3'] + dfArg_c['context1'].apply(fun,args=[l]).apply(fun1)

df['tweets']= df['response'] + df['context'].apply(fun,args=[l]).apply(fun1)
df1['tweets']= df1['response'] + df1['context'].apply(fun,args=[l]).apply(fun1)


dfArg_r['tweets']=dfArg_r['tweets'].apply(preprocess_text)
df['tweets']= df['tweets'].apply(preprocess_text) 
df1['tweets']=df1['tweets'].apply(preprocess_text)


In [14]:
df3=df[df['label']=='SARCASM']
df3

,label,response,context,labels,tweets
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...,0,@USER I do not get this .. obviously you do ca...
1,SARCASM,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...,0,@USER trying to protest about . Talking about ...
2,SARCASM,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...,0,@USER He makes an insane about of money from t...
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...,0,@USER Meanwhile Trump will not even release hi...
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...,0,@USER Pretty Sure the Anti-Lincoln Crowd Claim...
...,...,...,...,...,...
2495,SARCASM,"She's dangerous & self serving , but as an Ame...",[While @USER whitewashes #Palestine I'll remin...,0,"she is dangerous & self serving , but as an Am..."
2496,SARCASM,"@USER @USER I can't even say "" shame on you "" ...",[Activist parents jailed for #CitizenshipAmend...,0,"@USER I cannot even say "" shame on you "" to a ..."
2497,SARCASM,@USER @USER @USER So if someone was to murder ...,[Supreme Court leaves in place Kentucky aborti...,0,@USER So if someone was to murder you it would...
2498,SARCASM,This trilateral consultations aim to form a re...,[Russia meeting Taliban in Moscow and Tajikist...,0,This trilateral consultations aim to form a re...


In [15]:
df4=dfArg_r[df['label']=='SARCASM']
df4['labels']=df[df['label']=='SARCASM']['labels']
df4

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,response3,tweets,labels
0,0,@ user online don't get it.. think you don't e...,user online do not get it.. think you do not ...,0
1,1,@ user tries to protest again. talk about jake...,user tries to protest again. talk about jake ...,0
2,2,@ show the lives about madness about money lif...,show the lives about madness about money life...,0
3,3,@ user usually fails't remotely publish his de...,user usually fails't remotely publish his deg...,0
4,4,@ and pretty sure that the a - lincoln word of...,and pretty sure that the a - lincoln word of ...,0
...,...,...,...,...
2495,2495,"she is fair & far - serving, but as an angel n...","she is fair & far - serving, but as an angel n...",0
2496,2496,"@ user never can'scale even sign "" harm about ...","user never can'scale even sign "" harm about a...",0
2497,2497,"@ and so, saying chris was to murder you, woul...","and so, saying chris was to murder you, would...",0
2498,2498,"these trilateral systems aim to form, model an...","these trilateral systems aim to form, model an...",0


In [16]:
# df3=pd.concat(df['tweets'].values,dfArg_r['tweets'])

In [17]:
df['tweets']

0       @USER I do not get this .. obviously you do ca...
1       @USER trying to protest about . Talking about ...
2       @USER He makes an insane about of money from t...
3       @USER Meanwhile Trump will not even release hi...
4       @USER Pretty Sure the Anti-Lincoln Crowd Claim...
                              ...                        
4995    @USER You do not . I have purchased a lot on A...
4996    @USER #Emotions you say thinking face never kn...
4997    @USER You are so right ... " Yes ! #Silence is...
4998    @USER Another lazy delusional voter who takes ...
4999    @USER I hope you know no news outlet from Nige...
Name: tweets, Length: 5000, dtype: object

# Split train dataset into train, validation and test sets

In [18]:
def fun3(A):
  if len(A)<=3:
    return A
  else:
    return A[-1:-4:-1]

In [19]:

X_s = df['tweets'].values
xss= dfArg_r['tweets'].values
# xsss=df4['tweets'].values
X_s = np.concatenate((X_s, xss))
y_s = df['labels'].values
yss=df['labels'].values
# ysss=df4['labels'].values
y_s=arr = np.concatenate((y_s, yss))






Xt_s = df1['tweets'].values
yt_s = df1['labels'].values

train_text, temp_text, train_labels, temp_labels = X_s,Xt_s,y_s,yt_s

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)
val_text, test_text, val_labels, test_labels=temp_text, temp_text, temp_labels, temp_labels

# X_train, X_test, y_train, y_test = train_test_split(X_s, y_s, test_size=0.2, random_state=42)


In [20]:
# ak=pd.DataFrame(X_s,y_s)
# ak.to_csv('/content/drive/MyDrive/sarcasm-detection/argumented_data/context11111111111.csv')


In [21]:
X_s.size

10000

In [22]:
# pip install -U sentence-transformers

# Import BERT Model and BERT Tokenizer

---



---



In [23]:
# import BERT-base pretrained model
# bert = AutoModel.from_pretrained('bert-base-uncased')

# # Load the BERT tokenizer
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
# import BERT-base pretrained model


# from transformers import RobertaTokenizer, TFRobertaModel
# bert = RobertaModel.from_pretrained('roberta-base')

# # Load the BERT tokenizer
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

from transformers import RobertaTokenizer, TFRobertaModel
bert = RobertaModel.from_pretrained('roberta-large')

# Load the BERT tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
# sample data
text = ["this is a bert model tutorial", "we will fine-tune a bert model"]
# encode text
sent_id = tokenizer.batch_encode_plus(text, padding=True, return_token_type_ids=False)

In [25]:
# output
print(sent_id)

{'input_ids': [[0, 9226, 16, 10, 741, 2399, 1421, 35950, 2, 1, 1, 1], [0, 1694, 40, 2051, 12, 90, 4438, 10, 741, 2399, 1421, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


# Tokenization

In [26]:
# # get length of all the messages in the train set
# seq_len = [len(i.split()) for i in train_text]
# for i in train_text:
#   print(i)
# pd.Series(seq_len).hist(bins = 30)

In [27]:
max_seq_len = 180
# max_seq_len = 120

In [28]:
# total=0
# for i in range(10000):
#   total+=len(df['tweets'][i].split(' '))
# total/10000

In [29]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


# Convert Integer Sequences to Tensors

In [30]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

# Create DataLoaders

In [31]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 16

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

# Freeze BERT Parameters

In [32]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = True
# print(bert)

# Define Model Architecture

In [33]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      # self.fc1 = nn.Linear(1024,512)
      # self.fc2 = nn.Linear(512,2)
      self.fc1 = nn.Linear(1024,256)
      self.fc2 = nn.Linear(256,2)
      # dense layer 2 (Output layer)
      # self.fc3 = nn.Linear(128,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id,mask, return_dict=False)
      
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      # x = self.fc3(x)
      # apply softmax activation
      x = self.softmax(x)

      return x

In [34]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

In [35]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr =1e-5)

# Find Class Weights

In [36]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_wts = compute_class_weight('balanced', np.unique(train_labels), train_labels)

print(class_wts)

[1. 1.]


In [37]:
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 10

# Fine-Tune BERT

In [38]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in tqdm(enumerate(train_dataloader),total=10000//batch_size+1):
    # progress update after every 50 batches.
    # if step % 50 == 0 and not step == 0:
    #   print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()
    # my_lr_scheduler.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [39]:
# function for evaluating the model
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  # for step,batch in enumerate(val_dataloader):
  for step,batch in tqdm(enumerate(val_dataloader),total=10000//batch_size+1):
    # Progress update every 50 batches.
    # if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      # elapsed = format_time(time.time() - t0)
            
      # Report progress.
      # print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [40]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
!nvidia-smi

Wed Jul 21 02:35:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W /  70W |   2436MiB / 15109MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Start Model Training

In [ ]:
# set initial loss to infinite
best_valid_loss = float('inf')
acc=0
# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
epochs=10
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    for param_group in optimizer.param_groups:
        print('learning_rate- ',param_group['lr'])
    #train model
    train_loss,_ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    # if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), 'saved_weights.pt')


    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    pred=[]
    with torch.no_grad():
      for i in tqdm(range(0,90)):
        preds = model(test_seq[i*20:i*20+20].to(device), test_mask[i*20:i*20+20].to(device))
        preds = preds.detach().cpu().numpy()
        pred.append(preds)
    pred1=np.array(pred)
    pred2=pred1.reshape((1800,2))
    p=pd.DataFrame(pred2)
    p.to_csv('71_Rresponse.csv')
    pred3 = np.argmax(pred2, axis = 1)
    print(classification_report(test_y, pred3))
    print(accuracy_score(test_y, pred3))
    if accuracy_score(test_y, pred3)>acc:
      acc=max(accuracy_score(test_y, pred3),acc)
      p.to_csv(f'/content/drive/MyDrive/sarcasm-detection/Results/arg_twitter/{acc}_all_contest_16batch.csv')

    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 10
learning_rate-  1e-05




Evaluating...



              precision    recall  f1-score   support

           0       0.76      0.73      0.75       900
           1       0.74      0.77      0.76       900

    accuracy                           0.75      1800
   macro avg       0.75      0.75      0.75      1800
weighted avg       0.75      0.75      0.75      1800

0.7511111111111111

Training Loss: 0.521
Validation Loss: 0.526

 Epoch 2 / 10
learning_rate-  1e-05




Evaluating...



              precision    recall  f1-score   support

           0       0.70      0.90      0.79       900
           1       0.86      0.62      0.72       900

    accuracy                           0.76      1800
   macro avg       0.78      0.76      0.75      1800
weighted avg       0.78      0.76      0.75      1800

0.7566666666666667

Training Loss: 0.368
Validation Loss: 0.618

 Epoch 3 / 10
learning_rate-  1e-05


...


              precision    recall  f1-score   support

           0       0.79      0.67      0.72       900
           1       0.71      0.82      0.76       900

    accuracy                           0.75      1800
   macro avg       0.75      0.75      0.74      1800
weighted avg       0.75      0.75      0.74      1800

0.7455555555555555

Training Loss: 0.109
Validation Loss: 1.451

 Epoch 6 / 10
learning_rate-  1e-05




Evaluating...



              precision    recall  f1-score   support

           0       0.68      0.91      0.78       900
           1       0.87      0.57      0.69       900

    accuracy                           0.74      1800
   macro avg       0.77      0.74      0.74      1800
weighted avg       0.77      0.74      0.74      1800

0.7427777777777778

Training Loss: 0.084
Validation Loss: 1.576

 Epoch 7 / 10
learning_rate-  1e-05




Evaluating...



              precision    recall  f1-score   support

           0       0.80      0.65      0.71       900
           1       0.70      0.84      0.76       900

    accuracy                           0.74      1800
   macro avg       0.75      0.74      0.74      1800
weighted avg       0.75      0.74      0.74      1800

0.7416666666666667

Training Loss: 0.077
Validation Loss: 1.592

 Epoch 8 / 10
learning_rate-  1e-05


# Load Saved Model

In [ ]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

# Get Predictions for Test Data

In [ ]:
pred=[]
with torch.no_grad():
  for i in range(0,90):
    preds = model(test_seq[i*20:i*20+20].to(device), test_mask[i*20:i*20+20].to(device))
    preds = preds.detach().cpu().numpy()
    pred.append(preds)
print(np.array(pred).shape)

In [ ]:
pred1=np.array(pred)
# pred1

In [ ]:
pred2=pred1.reshape((1800,2))
pred2.shape

In [ ]:
p=pd.DataFrame(pred2)
p.to_csv('context_2.csv')

In [ ]:
pred3 = np.argmax(pred2, axis = 1)
pred3 

In [ ]:
print(classification_report(test_y, pred3))

In [ ]:
# confusion matrix
# print(pred)
pd.crosstab(test_y, pred3)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

print(accuracy_score(test_y, pred3))

In [ ]:
import scipy

In [ ]:
brcc=pd.read_csv('76.6_roberta_large_100_32_2e-5_4epoch_preprocessing+@reply+user.csv')
brcc=np.array(brcc[['0','1']])
brcc=scipy.special.softmax(brcc,axis=1)
# bsvm=np.multiply(bsvm,-1)
brcc

In [ ]:
brc=pd.read_csv('71_Rresponse.csv')
brc=np.array(brc[['0','1']])
brc=scipy.special.softmax(brc,axis=1)
# bsvm=np.multiply(bsvm,-1)
brc

In [ ]:
br=pd.read_csv('77.7_2context+response_batch16+4epoch_190maxlen.csv')
br=np.array(br[['0','1']])
br=scipy.special.softmax(br,axis=1)
# bsvm=np.multiply(bsvm,-1)
br

In [ ]:
bc=pd.read_csv('76.56_2context_+response_temserflow.csv')
bc=np.array(bc[['0','1']])
bc=scipy.special.softmax(bc,axis=1)
# bsvm=np.multiply(bsvm,-1)
bc

In [ ]:
bkc=pd.read_csv('75_lstm_base_response.csv')
bkc=np.array(bkc[['0','1']])
bkc=scipy.special.softmax(bkc,axis=1)
# bsvm=np.multiply(bsvm,-1)
bkc

In [ ]:
# bertc=np.array(bc[['0','1']])
bertc1=np.multiply(bc,0.6)

# bertrcc=np.array(brcc[['0','1']])
bertrcc1=np.multiply(brcc,0.8)

# bertcr=np.array(brc[['0','1']])
bertcr1=np.multiply(brc,0.6)

# bertr=np.array(br[['0','1']])
bertr1=np.multiply(br,0.45)



# # bertc=np.array(bc[['0','1']])
# bertc1=np.multiply(bc,0.4)

# # bertrcc=np.array(brcc[['0','1']])
# bertrcc1=np.multiply(brcc,0.8)

# # bertcr=np.array(brc[['0','1']])
# bertcr1=np.multiply(brc,0.4)

# bertr=np.array(br[['0','1']])
# bertr1=np.multiply(br,0.8)

In [ ]:
yhats = [bertc1,bertr1,bertcr1,bertrcc1]
# yhats = [bertcr1,bertrcc1,bertr1]
# yhats = [bertcr1,bertrcc1,bertr1]
# import numpy as np
# yhats = np.array(yhats)

# # sum across ensemble members
# summed = np.sum(yhats, axis=0)
# # argmax across classes
# result = np.argmax(summed, axis=1)

# from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# accuracy_score(result,test_y)

In [ ]:
ans=[]
yhat=[bc,br,brcc,brc,bkc]
# yhat=[brcc,br,brc]
for i in yhat:
  ans.append(np.argmax(i, axis=1))
summed = np.sum(ans, axis=0)/4
res=[]
for i in summed:
  if i<=0.5:
    res.append(0)
  else:
    res.append(1)

print(res)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# accuracy_score(res,test_y)


print(classification_report(test_y, res))
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

print(accuracy_score(test_y, res))